In [1]:
import sys; sys.path.append('src')
import src.utils.callable as C
from src.utils import SingleWorkerDataset, Dataloader

In [2]:
path = ['data/k400/test.tar', 'data/k400/test1.tar']

In [5]:
transforms = [
    C.SampleFrames(128, 'uniform'),
    C.FramesToArray(),
]

dl = Dataloader(path, batch_size=32, transforms=transforms, num_workers=8, prefetch_count=8)

In [6]:
from tqdm import tqdm

for batch in tqdm(dl):
    pass

14it [00:52,  2.37s/it]Process ForkProcess-15:
Process ForkProcess-11:
Process ForkProcess-14:
Process ForkProcess-12:
Process ForkProcess-13:
Process ForkProcess-16:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/lei/miniconda3/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/lei/miniconda3/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/home/lei/miniconda3/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/home/lei/miniconda3/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/home/lei/miniconda3/lib/python3.10/multiprocessing/process.py", line 314, in _bootstrap
    self.run()
  File "/home/lei/miniconda3/lib/python3.10/multiprocessing/pro

KeyboardInterrupt: 

In [ ]:
# 1.54